In [17]:
from flask import Flask, request, render_template, url_for, jsonify
import os
import shutil
from data_utils.utils_function import *
from data_utils.get_planet_scenes import search, download_tif_files
import json
import collections

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
collections.Iterable = collections.abc.Iterable
from deploy_model_instances import *
from data_handler import *


class MarineDebrisApp:
    def __init__(self):
        self.app = Flask(__name__)
        self.MAX_CLOUD_PERCENTAGE = 0.5
        self.SEARCH_FOLDER = "searches/2023-05-13_19-21-56.487285"
        self.model = ModelDeployment(model_weights_path="best_model_efficientnetb4_4.h5")
        self.results_list = []
        self.app.route("/")(self.home)
        self.app.route("/Dashboard")(self.dashboard)
        self.app.route('/upload_geojson', methods=['POST'])(self.upload_geojson)
        self.app.route('/get_images')(self.get_images)
        self.app.route('/predict', methods=['POST','GET'])(self.predict)

    def home(self):
        return render_template("index.html", css_file=url_for('static', filename='mapbox-gl-draw.css'),
                               js_file=url_for('static', filename='mapbox-gl-draw.js'))

    def dashboard(self):
        return render_template("dashboard.html")

    def upload_geojson(self):
        posted_file = str(request.files['geojsonFile'].read(), 'utf-8')
        posted_file = json.loads(posted_file)
        geometry_json, start_date_time_formatted, end_date_time_formatted = \
            extract_geometry_timerange_from_json(posted_file)
        print(geometry_json)
        print(start_date_time_formatted)
        print(end_date_time_formatted)
        q_res = search(geometry=geometry_json, start_date=start_date_time_formatted
                     , end_date=end_date_time_formatted, cc=self.MAX_CLOUD_PERCENTAGE)
        self.SEARCH_FOLDER = download_tif_files(q_res)
        print("files were downloaded to {}".format(self.SEARCH_FOLDER))
        return render_template("dashboard.html", search_path=self.SEARCH_FOLDER)

    def get_images(self):
        source_dir = os.path.join("static", "predictions").replace("\\", "/")
        target_dir = 'static/images'

        # Copy image files from the source directory to the target directory
        for filename in os.listdir(source_dir):
            source_path = os.path.join(source_dir, filename)
            target_path = os.path.join(target_dir, filename)
            shutil.copy2(source_path, target_path)

        # Get the list of image files in the target directory
        image_files = [f for f in os.listdir(target_dir)
                       if os.path.isfile(os.path.join(target_dir, f))
                       and '.jpg' in f]
        image_urls = [{'image_url': f'/static/images/{f}', 'image_name': f} for f in image_files]
        return jsonify(image_urls)

    def predict(self):
        model_input = data_load(self.SEARCH_FOLDER)
        self.results_list = self.model.execute_job(input_data=model_input)
        predictions_path = os.path.join("static", "predictions").replace("\\", "/")
        writing_results(self.results_list, predictions_path)
        print("predicting")
        return self.dashboard()

    def run(self):
        self.app.run()

In [18]:
marine_debris_app = MarineDebrisApp()
marine_debris_app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [14/May/2023 03:22:58] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2023 03:22:58] "GET /static/mapbox-gl-draw.css HTTP/1.1" 304 -
127.0.0.1 - - [14/May/2023 03:22:58] "GET /static/mapbox-gl-draw.js HTTP/1.1" 304 -
127.0.0.1 - - [14/May/2023 03:23:00] "GET /Dashboard HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2023 03:23:00] "GET /static/loading_animation.gif HTTP/1.1" 304 -
127.0.0.1 - - [14/May/2023 03:23:00] "GET /get_images HTTP/1.1" 200 -
127.0.0.1 - - [14/May/2023 03:23:00] "GET /static/images/tile_12_8.jpg HTTP/1.1" 304 -
127.0.0.1 - - [14/May/2023 03:23:00] "GET /static/images/tile_12_8prediction.jpg HTTP/1.1" 304 -
127.0.0.1 - - [14/May/2023 03:23:00] "GET /static/images/tile_22_4.jpg HTTP/1.1" 304 -
127.0.0.1 - - [14/May/2023 03:23:00] "GET /static/images/tile_22_4prediction.jpg HTTP/1.1" 304 -
127.0.0.1 - - [14/May/2023 03:23:00] "GET /static/images/tile_22_6.jpg HTTP/1.1" 304 -
127.0.0.1 - - [14/May/2023 03:23